In [22]:
# import libraries needed for the project

#data handling tools
import pandas as pd
import numpy as np

#data visualization tools
import matplotlib.pyplot as plt
import seaborn as sns

#machine learning tools
# TensorFlow
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras import models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import TextVectorization, Embedding

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# other libraries
import random


In [2]:
# import the training and testing data
train_df = pd.read_csv('./Data/train.csv')
test_df = pd.read_csv('./Data/test.csv')

In [3]:
# read the head of the dataset to become familiar with the data
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


since we're not working with a sequential data, we might want to shuffle to prevent the models from learning in some order

In [4]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [5]:
# looking at the test set
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
# get 5 random tweets and their classes
random_index = random.randint(0, len(train_df_shuffled)-5)
for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f'Target: {target}', '(real disaster)' if target > 0 else '(not real disaster)')
  print(f'Text:\n{text}\n')
  print('---')

Target: 0 (not real disaster)
Text:
Marquei como visto Dragon Ball Super - 1x1 - The God of Destruction\'s Dream http://t.co/vJLnsKbG86 #bancodeseries

---
Target: 1 (real disaster)
Text:
{:en}Tension In Bayelsa As Patience Jonathan Plans To Hijack APCåÊPDP{:} http://t.co/W1ufeibALa http://t.co/bIULQpRxke

---
Target: 0 (not real disaster)
Text:
The #Bible sometimes backs up the truck and unloads a descriptive deluge of indecency on us. @chuckswindoll explains why on @iflcanada next.

---
Target: 0 (not real disaster)
Text:
RT @SleepJunkies: Sleeping pills double your risk of a car accident http://t.co/7s9Nm1fiCT

---
Target: 0 (not real disaster)
Text:
RT: Geek_Apocalypse: 4pm GMT :Hesse plays dark souls 2 day 9: http://t.co/TnGPsHNL87 http://t.co/imzLNZLtF5 #etcPB

---


In [7]:
# Let's split the data into training and validation dataset
X_train, X_val, y_train, y_val = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                    train_df_shuffled['target'].to_numpy(),
                                                    test_size=0.1, random_state=42)

In [8]:
# check for sizes of the split
len(X_train), len(X_val), len(y_train), len(y_val)

(6851, 762, 6851, 762)

In [9]:
# visualise the training data
X_train[:11], y_val[0:11]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt',
        'Lava Dragon Breeder! I j

## Text Preprocessing

In [10]:
# check for the average number of tokens in each tweet
avg_token = round(sum([len(i.split()) for i in train_df_shuffled['text']])/len(train_df_shuffled['text']))

In [11]:
# Hyperparameters
max_vocab_length = 10000
max_length = avg_token

# create an instance of the TextVectorization Layer
text_vectorizer = TextVectorization(
    max_tokens = max_vocab_length,
    output_mode = 'int',
    output_sequence_length = max_length
)

In [12]:
# adapt the tokenizer to the training samples
text_vectorizer.adapt(X_train)

In [13]:
# Get an embedding layer to accurately find the relationship among the tokens
embedding = Embedding(
    input_dim = max_vocab_length,
    output_dim = 128,
)

In [14]:
# get the embedding of a random tweet
random_tweet = random.choice(X_train)
print(f'Original tweet:\n{random_tweet}\
        \n\nEmbedded version:')
embedding(text_vectorizer([random_tweet]))

Original tweet:
Police Officer Wounded Suspect Dead After Exchanging Shots: Richmond police officer wounded suspect killed a... http://t.co/w0r8EAOKA0        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03085278,  0.01825849, -0.03189526, ...,  0.04406916,
          0.0134852 ,  0.046249  ],
        [-0.02869268,  0.03961558,  0.02418334, ..., -0.04137362,
          0.03559723, -0.01131426],
        [ 0.02818639,  0.00515186,  0.03736362, ..., -0.022128  ,
         -0.00468047,  0.04613972],
        ...,
        [ 0.00995297, -0.02242906, -0.00883968, ...,  0.04375359,
          0.00627406, -0.00459325],
        [-0.00454586,  0.00584998, -0.0183364 , ..., -0.03504889,
         -0.0434695 ,  0.0148335 ],
        [ 0.04274435,  0.02362743,  0.00693654, ...,  0.0063723 ,
          0.01986934,  0.01774216]]], dtype=float32)>

## Helper functions

In [28]:
# a list to store all scores so that a dataframe can be created to effectively compare models against the baseline
model_score_list = []

In [26]:
# A function to take a model name, The predicted values and the actual values and return scores
def model_scores(model_name:str, y_preds:list, y_true) -> dict:
  accuracy = accuracy_score(y_true, y_preds)
  precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_preds, average='weighted')
  results = {
      'model_name': model_name,
      'accuracy': accuracy,
      'precision': precision,
      'recall': recall,
      'f1': f1
  }
  model_score_list.append(results)
  return results

## Baseline Modelling
For the baseline, we will use a traditional model which in this case is the `naive bayes` classifier

In [17]:
 # create a pipeline for the model
 model_0 = Pipeline([
  ('tfidf', TfidfVectorizer()),
  ('clf', MultinomialNB())
 ])


 # fit the model
 model_0.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [19]:
# evaluate the model
baseline_score = model_0.score(X_val, y_val)
print(f"The baseline Model acheives an accuracy of {baseline_score*100:.2f}%")

The baseline Model acheives an accuracy of 79.27%


In [21]:
# make some predictions with the baseline model
model_0_preds = model_0.predict(X_val)
model_0_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [29]:
# Get the scores for the baseline model
print(model_scores('MultinomialNB', model_0_preds, y_val))

{'model_name': 'MultinomialNB', 'accuracy': 0.7926509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}
